<img style="float: center;" src="images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Ghani, Rayid, Frauke Kreuter, Julia Lane, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Brian Kim, Avishek Kumar, Jonathan Morgan, Ursula Kaczmarek.


# Dataset Exploration
----------
Basic dataset exploration

## Table of Contents
- [Introduction](#Introduction)
    - [Learning objectives](#Learning-objectives)
    - [Methods](#Methods)


- [Python setup](#Python-Setup)


- [Load the data](#Load-the-Data)
    - [Establish a Connection to the Database](#Establish-a-Connection-to-the-Database)
    - [Formulate Data Query](#Formulate-Data-Query)
    - [Pull Data from the Database](#Pull-Data-from-the-Database)
    

- [Using Python and SQL](#Using-Python-and-SQL)
    - [What is in the Database](#What-is-in-the-Database)
   
   
- [Summary Statistics](#Summary-Statistics)
    - [Query time and the PostgreSQL EXPLAIN function](#Query-time-and-the-PostgreSQL-EXPLAIN-function)
    - [Exploring education and training data](#Exploring-education-and-training-data)
    - [Ohio data](#Ohio-data)
    - [Employment and Education](#Employment-and-Education)
    

- [Creating New Measures](#Creating-New-Measures)

## Introduction
- Back to [Table of Contents](#Table-of-Contents)

In an ideal world, we will have all of the data we want with all of the desirable properties (no missing values, no errors, standard formats, and so on). 
However, that is hardly ever true - and we have to work with using our datasets to answer questions of interest as intelligently as possible. 

In this notebook, we will discover the datasets we have on the ADRF, and we will use our datasets to answer some questions of interest. 

### Learning Objectives
- Back to [Table of Contents](#Table-of-Contents)

This notebook will give you the opportunity to spend some hands-on time with the data. 

You will have an opportunity to explore the different datasets in the ADRF, and this notebook will take you around the different ways you can analyze your data. This involves looking at basic metrics in the larger dataset, taking a random sample, creating derived variables, making sense of the missing values, and so on. 

This will be done using both SQL and `pandas` (a Python package). The `sqlalchemy` Python package provides a connetion to the database to pull data into Python. 

This notebook will provide an introduction and examples for: 

- How to create new tables from the larger tables in database (sometimes called the "analytical frame")
- How to explore different variables of interest
- How to explore aggregate metrics
- How to handle missing values
- How to join newly created tables

### Methods
- Back to [Table of Contents](#Table-of-Contents)

We will be using the `sqlalchemy` Python package to access tables in our database server - PostgreSQL. 

To read the results of our queries, we will be using the `pandas` Python package, which has the ability to read tabular data from SQL queries into a pandas DataFrame object. Within `pandas`, we will use various commands:

- Subsetting data
- `groupby`
- `merge`

Within SQL, we will use various queries to:

- select data subsets
- Sum over groups
- create new tables
- Count distinct values of desired variables
- Order data by chosen variables
- Select a random sub-sample

## Python Setup
- Back to [Table of Contents](#Table-of-Contents)

In Python, we `import` packages. The `import` command allows us to use libraries created by others in our own work by "importing" them. You can think of importing a library as opening up a toolbox and pulling out a specific tool. Among the most famous Python packages:
- `numpy` is short for "numerical Python". `numpy` is a lynchpin in Python's scientific computing stack. Its strengths include a powerful *N*-dimensional array object, and a large suite of functions for doing numerical computing. 
- `pandas` is a library in Python for data analysis that uses the DataFrame object (modeled after R DataFrames, for those familiar with that language) which is similiar to a spreedsheet but allows you to do your analysis programaticaly rather than the point-and-click of Excel. It is a lynchpin of the PyData stack and is built on top of `numpy`.  
- `sqlalchemy` is a Python library for interfacing with a PostGreSQL database. 

In [ ]:
# pandas-related imports
import pandas as pd

# database interaction imports
import sqlalchemy

__When in doubt, use shift + tab to read the documentation of a method.__

__The `help()` function provides information on what you can do with a function.__

In [ ]:
# for example
help(sqlalchemy.create_engine)

## Load the Data

- Back to [Table of Contents](#Table-of-Contents)

We can execute SQL queries using Python to get the best of both worlds. For example, Python - and pandas in particular - make it much easier to calculate descriptive statistics of the data. Additionally, as we will see in the Data Visualization exercises, it is relatively easy to create data visualizations using Python. 

Pandas provides many ways to load data. It allows the user to read the data from a local csv or excel file, pull the data from a relational database, or read directly from a URL (when you have internet access). Since we are working with the PostgreSQL database `appliedda` in this course, we will demonstrate how to use pandas to read data from a relational database. For examples to read data from a CSV file, refert to the pandas documentation [Getting Data In/Out](pandas.pydata.org/pandas-docs/stable/10min.html#getting-data-in-out).

The function to run a SQL query and pull the data into a pandas dataframe (more to come) is `pd.read_sql()`. Just like doing a SQL query from pgAdmin, this function will ask for some information about the database, and what query you would like to run. Let's walk through the example below.

### Establish a Connection to the Database
- Back to [Table of Contents](#Table-of-Contents)

The first parameter is the connection to the database. To create a connection we will use the SQLAlchemy package and tell it which database we want to connect to, just like in pgAdmin. Additional details on creating a connection to the database are provided in the [Databases](02_1_Databases.ipynb) notebook.

__Database Connection__

In [ ]:
# to create a connection to the database, 
# we need to pass the name of the database and host of the database

host = 'stuffed.adrf.info'
DB = 'appliedda'

connection_string = "postgresql://{}/{}".format(host, DB)
conn = sqlalchemy.create_engine(connection_string)

> Note we can parameterize Python `string` objects - using the built-in `.format()` function. We will use various formulations in the program notebooks (eg when building queries), some examples are:
1. Empty brackets (shown above) which simply inserts the variable in the string; when there is more than one set of brackets Python will insert variables in the order they are listed
2. Brackets with formatting can be used to make print statements more readable (eg `'text with formatted number with comma and 1-digit decimal {:,.1f}'.format(number_value)` will print `123,456.7` instead of `123456.7123401`)
3. Named brackets to use the same variables multiple times in a text block (we use this in more compicated queries eg when creating "labels" and "features" for Machine Learning models)

### Formulate Data Query
- Back to [Table of Contents](#Table-of-Contents)


__create a query as a `string` object in Python__

In [ ]:
# query to pull data on general studies program graduates in 2014
query = '''
SELECT *
FROM mo_dhe.completions
WHERE calyear = 2014 AND progone = '240102'
LIMIT 20
'''

Note:

- the three quotation marks surrounding the query body is called multi-line string. It is quite handy for writing SQL queries because the new line character will be considered part of the string, instead of breaking the string

In [ ]:
# Now that we have defined a variable `query`, we can call it in the code
print(query)

> Note that the `LIMIT` provides one simple way to get a "sample" of data. However, using `LIMIT` does **not provide a _random_** sample; it is just based on what is fastest for the database to return.

### Pull Data from the Database
- Back to [Table of Contents](#Table-of-Contents)

Now that we have the two parameters (database connection and query), we can pass them to the `pd.read_sql()` function, and obtain the data.

In [ ]:
# here we pass the query and the connection to the pd.read_sql() function 
df = pd.read_sql(query, conn)

In [ ]:
# the first five rows of our Missouri grads pandas dataframe
df.head()

## Using Python and SQL
- Back to [Table of Contents](#Table-of-Contents)

__What are different possible measures of employment for Missouri graduates?__

To explore possible metrics, we will need to combine education and employment (in our case, UI wage records) data. We will start slow and explore these datasets individually, then work up to some initial metrics of employment.

### What is in the Database
- Back to [Table of Contents](#Table-of-Contents)

As introduced in the [Databases](./02_1_Databases.ipynb) notebook, there are a few different ways to connect and explore the data in the database. 

__ Schemas, Tables, and Columns in database__

Let's pull the list of schema names in the database, the list of tables in these schemas and the list of columns in these tables.

In [ ]:
# See all available schemas:
query = '''
SELECT schema_name 
FROM information_schema.schemata;
'''
pd.read_sql(query, conn)

> As a reminder, in this class you have access to the following schemas: 'public', 'data_ohio_olda_2018', 'il_des_kcmo', 'kcmo_lehd', 'mo_dhe', 'mo_doc', 'mo_dolir', 'ada_edwork' and your team schema ('ada_edwork_#', where the # is your team number)

In [ ]:
schemas = """
'public', 'data_ohio_olda_2018', 'il_des_kcmo', 'kcmo_lehd', 'mo_dhe', 'mo_doc', 'mo_dolir',
'ada_edwork', 'in_data_2019'
"""

In [ ]:
# confirm our schemas exist with 
# an updated version of the previous query
query = '''
SELECT schema_name 
FROM information_schema.schemata
WHERE schema_name IN ({})
'''.format(schemas)
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT schemaname, tablename
FROM pg_tables
WHERE schemaname IN ({})
'''.format(schemas)

tables = pd.read_sql(query, conn)
# print tables not in the public schema
print(tables.query("schemaname != 'public'"))

In [ ]:
# list all the tables in the Missouri education schema:
sorted(tables[tables["schemaname"] == 'mo_dhe']['tablename'])

> Note the two ways shown above to subset a `Pandas.DataFrame`:
1. Use the built-in `.query()` function
2. Create an array of `True` and `False` values (done in this line: `tables["schemaname"] == 'mo_dhe'`)

In [ ]:
# We can look at column names within tables
# here we'll set the schema and table with variables

schema = 'mo_dhe'
tbl = 'completions'

query = '''
SELECT * 
FROM information_schema.columns 
WHERE table_schema = '{}' AND table_name = '{}'
'''.format(schema, tbl)

# read and print results
pd.read_sql(query, conn)

__Missouri education data__:
- `mo_dhe.enrollments`: individual Missouri college/university enrolled student data
- `mo_dhe.completions`: individual Missouri college/university graduate data


In [ ]:
query = '''
SELECT *
FROM mo_dhe.enrollments
limit 100;
'''
df = pd.read_sql(query, conn)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
query = '''
SELECT *
FROM mo_dhe.completions
limit 100;
'''
df = pd.read_sql(query, conn)

In [ ]:
df.head()

In [ ]:
df.info()

## Summary Statistics
- Back to [Table of Contents](#Table-of-Contents)

In this section, let's start looking at aggregate statistics on the data. 

Let's explore a few specific questions to better understand our data:
- How are the data distributed across credential programs?
- How do quarterly wages compare across industries?

> Note: __ Large tables__ can take a long time to process on shared databases. The MO wage table as over 133.4M records, so we will demonstrate using SQL and Python with consideration for how much data we are reading back into Python

In [ ]:
# additionally we'll use the Python time package
# to see how long different queries takes to return

import time # import time package

In [ ]:
start_time = time.time() # get current time

query = """
SELECT count(*) 
FROM kcmo_lehd.mo_wage
WHERE year = 2015 AND quarter = 1
"""
# print results
print(pd.read_sql(query, conn)) 
# print analysis time
print('Query returned in {:.1f} seconds'.format(time.time()-start_time))

In [ ]:
# example count of records for IL 2015 Q1

start_time = time.time() # get current time

query = """
SELECT count(*) 
FROM il_des_kcmo.il_wage         
WHERE year = 2015 AND quarter = 1
"""
# print results
print(pd.read_sql(query, conn)) 
# print analysis time
print('Query returned in {:.1f} seconds'.format(time.time()-start_time))

In [ ]:
# example count      
start_time = time.time() # get current time

query = """
SELECT count(*) 
FROM data_ohio_olda_2018.oh_ui_wage_by_employer
WHERE year = 2015 AND quarter = 1
"""
# print results
print(pd.read_sql(query, conn))
# print analysis time
print('Query returned in {:.1f} seconds'.format(time.time()-start_time))

In [ ]:
# example count of records for 2015 Q1

start_time = time.time() # get current time

query = """
SELECT count(*) 
FROM in_data_2019.wages_by_employer
WHERE year = 2007 AND quarter = 2
"""
# print results
print(pd.read_sql(query, conn)) 
# print analysis time
print('Query returned in {:.1f} seconds'.format(time.time()-start_time))

> A **question to consider**: This simple count is one measure of the total jobs in 2015 Q1. What may we want to consider when defining a "job" in addition to just being a row in this dataset?

In [ ]:
# the easiest way to look at a small subset of records is what
# is already demonstrated above: simply add a LIMIT clause
# again we'll look at how long this query takes to return

# get current time
start_time = time.time()

query = '''
SELECT *
FROM kcmo_lehd.mo_wage
WHERE year = 2015 AND quarter = 1
LIMIT 20;
'''
# get results
df_earnings = pd.read_sql(query, conn)
# print analysis time
print('Query returned in {:.1f} seconds'.format(time.time()-start_time))

In [ ]:
df_earnings.head()

In [ ]:
df_earnings.columns

> Reminder: you can refer to the documentation for more information on each dataset

In [ ]:
# we can get descriptive stats from the DataFrame:
df_earnings.describe(include='all')

In [ ]:
# as with the 'count` example, we will get basic stats
# from the database using SQL; eg the wage distribution

# first, just the 25th percentile value
query = """
SELECT percentile_cont(0.25)
    WITHIN GROUP (ORDER BY wage)
FROM kcmo_lehd.mo_wage
WHERE year = 2015 AND quarter = 1
"""
# display result
pd.read_sql(query, conn)

> Note: as of Jan 1, 2015, the minimum wage in MO was \\$7.65 per hour. Assuming a 35 hour work-week and 12 weeks in a quarter, someone working an entire quarter at minumum wage would earn \\$3,213 in the quarter (ignoring taxes).

> Minimum wage in Illinois in 2015 was \\$10.00 per hour.

> Minimum wage in Ohio in 2015 was \\$8.10 per hour (\\$4.05 for tipped employees).

> As of Jan 1, 2015, minimum wage in Indiana was \\$7.25 per hour.

In [ ]:
# use a list of percentile values at which to show the earnings value
query = """
SELECT percentile_cont(array[0.1, 0.25, 0.5, 0.75, 0.9])
WITHIN GROUP (ORDER BY wage)
FROM kcmo_lehd.mo_wage
WHERE year = 2015 AND quarter = 1
"""
# display result
pd.read_sql(query, conn)

In [ ]:
# values above return in a single cell, 
# add "unnest" to get values in a single cell
# also add a reference column and column names

start_time = time.time()

query = """
SELECT unnest(
        percentile_cont(array[0.1, 0.25, 0.5, 0.75, 0.9])
        WITHIN GROUP (ORDER BY wage)
    ) AS earnings_value,
    unnest(array[0.1, 0.25, 0.5, 0.75, 0.9]) AS percentile_value
FROM kcmo_lehd.mo_wage
WHERE year = 2015 AND quarter = 1
"""
# get the result
df = pd.read_sql(query, conn)
# print analysis time
print('Query returned in {:.1f} seconds'.format(time.time()-start_time))
# view result
df

One way to characterize a job is the employer industry. Note the minor differences in how the wage tables are presented from the different states: for both Illinois and Missouri, we have a different table for employers and for jobs (person <-> employer combinations) and we need to get the NAICS code from the `<st>_qcew_employers` table to calculate industry earnings. In Ohio and Indiana, we do not have information about the employers beyond what is presented in the wage record data.

In [ ]:
# check how many records from our Earnings data matches the Employers data
# just for 2015 Q1 data

start_time = time.time()

query = """
SELECT count(*) 
FROM kcmo_lehd.mo_wage AS earn
JOIN kcmo_lehd.mo_qcew_employers AS empl
ON earn.ein = empl.ein 
WHERE earn.year = 2015 AND earn.quarter = 1 
    AND empl.year = 2015 AND empl.qtr = 1
"""
res = pd.read_sql(query, conn)
print('query took {:.1f} seconds'.format(time.time()-start_time))
print(res)

In [ ]:
# view number of wage records by NAICS code for 2015 Q1

start_time = time.time()

query = """
SELECT empl.naics, count(*) 
FROM kcmo_lehd.mo_wage AS earn
JOIN kcmo_lehd.mo_qcew_employers AS empl
ON earn.ein = empl.ein 
WHERE earn.year = 2015 AND earn.quarter = 1 
    AND empl.year = 2015 AND empl.qtr = 1
GROUP BY naics
LIMIT 10
"""
res = pd.read_sql(query, conn)
print('query took {:.1f} seconds'.format(time.time()-start_time))
print(res)

In [ ]:
# for the Ohio data
start_time = time.time()

query = """
SELECT naics_3_digit, count(*)
FROM data_ohio_olda_2018.oh_ui_wage_by_employer
WHERE year = 2015 AND quarter = 1
GROUP BY naics_3_digit
LIMIT 10
"""
# get the result
df = pd.read_sql(query, conn)
# print analysis time
print('Query returned in {:.1f} seconds'.format(time.time()-start_time))
# view result
df

> Note that OH and IN have 3-digit codes. The industry descriptions can be looked up in the `appliedda` database table `public.naics_<year>` (lookup tables are included for `year`s 2002, 2007, 2012, 2017)

In [ ]:
# for the Indiana data
start_time = time.time()

query = """
SELECT naics_3_digit, count(*)
FROM in_data_2019.wages_by_employer
WHERE year = 2015 AND quarter = 1
GROUP BY naics_3_digit
LIMIT 10
"""
# get the result
df = pd.read_sql(query, conn)
# print analysis time
print('Query returned in {:.1f} seconds'.format(time.time()-start_time))
# view result
df

### Query time and the PostgreSQL EXPLAIN function
[Table of Contents](#Table-of-Contents)

As you begin creating more advanced or complicated queries, it is useful to have a sense of how long different queries take. One way is to simple record and observe how long different queries take, as has been done in the notebook so far.

You may get a sense of when a query is taking a long time. Slow queries could be the result of many people using the database at the same time, the database or table you are using are being `VACUUM`ed, or because of a poorly formed query or data structure (eg not making use of or having `indexes` on columns used to frequently subset or match the data)

> **The PostgreSQL "Explain" function** can help determine in what order the query is executed and whether it is making use of indexes. It is a little difficult to interpret, but here is an exmaple using the above Missouri query. The way to read this is start at the bottom of the RESULT and read up (steps outlined below the query output)
>   
    -- QUERY:
    EXPLAIN 
    SELECT count(*) 
    FROM kcmo_lehd.mo_wage AS earn
    JOIN kcmo_lehd.mo_qcew_employers AS empl
    ON earn.ein = empl.ein 
    WHERE earn.year = 2015 AND earn.quarter = 1 
        AND empl.year = 2015 AND empl.qtr = 1
>  
    -- RESULT
    Aggregate  (cost=1513710.83..1513710.84 rows=1 width=0)
      ->  Merge Join  (cost=1398108.92..1498397.93 rows=6125163 width=0)
            Merge Cond: (empl.ein = earn.ein)
            ->  Sort  (cost=554856.39..555314.27 rows=183152 width=65)
                  Sort Key: empl.ein
                  ->  Bitmap Heap Scan on mo_qcew_employers empl  (cost=90225.73..531331.95 rows=183152 width=65)
                        Recheck Cond: ((year = 2015) AND (qtr = 1))
                        ->  BitmapAnd  (cost=90225.73..90225.73 rows=183152 width=0)
                              ->  Bitmap Index Scan on mo_qcew_employers_year_index  (cost=0.00..23216.37 rows=754125 width=0)
                                    Index Cond: (year = 2015)
                              ->  Bitmap Index Scan on mo_qcew_employers_quarter_index  (cost=0.00..66917.53 rows=2173479 width=0)
                                    Index Cond: (qtr = 1)
            ->  Materialize  (cost=843252.53..858244.13 rows=2998321 width=65)
                  ->  Sort  (cost=843252.53..850748.33 rows=2998321 width=65)
                        Sort Key: earn.ein
                        ->  Append  (cost=0.00..397717.80 rows=2998321 width=65)
                              ->  Seq Scan on mo_wage earn  (cost=0.00..0.00 rows=1 width=32)
                                    Filter: ((year = 2015) AND (quarter = 1))
                              ->  Seq Scan on mo_wage_2015q1 earn_1  (cost=0.00..397717.80 rows=2998320 width=65)
                                    Filter: ((year = 2015) AND (quarter = 1))

So, this query runs in the following steps:
1. scans the employer year and quarter indexes
2. returns only rows where year and quarter meet our criteria
3. sorts the result based on our identifier column
4. performs same 3 steps on the earnings data (scan, filter, sort)
5. merges the two tables based on the identifier column
6. counts the rows (the final, "Aggregate" step)

For example, try running explain on a slightly different variant of the previous query:

    EXPLAIN SELECT count(*) 
    FROM il_des_kcmo.il_wage AS earn
    JOIN il_des_kcmo.il_qcew_employers AS empl
    ON earn.ein = empl.ein 
        AND earn.seinunit = empl.seinunit
        AND earn.empr_no = empl.empr_no
        AND earn.year = empl.year
        AND earn.quarter = empl.quarter

In [ ]:
# let's peruse a sample of Missouri data

start_time = time.time()

query = """
SELECT earn.year, earn.quarter, earn.wage, empl.naics
FROM kcmo_lehd.mo_wage AS earn
JOIN kcmo_lehd.mo_qcew_employers AS empl
ON earn.ein = empl.ein 
WHERE earn.year = 2015 AND earn.quarter = 1 
    AND empl.year = 2015 AND empl.qtr = 1
LIMIT 100
"""
res = pd.read_sql(query, conn)

# report how long it took to pull data
print('query took {:.1f} seconds'.format(time.time()-start_time))

In [ ]:
res.head()

In [ ]:
res.describe()

In [ ]:
# what is the distribution of earnings in our sample?
res['wage'].describe(percentiles=[0.1,0.25,0.5, 0.75, 0.9])

In [ ]:
# and earnings by industry?
res.groupby('naics')['wage'].describe(percentiles=[0.1,0.25,0.5, 0.75, 0.9])

In [ ]:
# now we'll query the database for earnings distribution by 2-digit NAICS in Q1 2015

start_time = time.time()

query="""
SELECT naics2, 
    unnest(percentile_cont(array[0.1,0.25,0.5, 0.75, 0.9]) 
    within group (ORDER BY wage)) AS earnings,
    unnest(array[0.1,0.25,0.5, 0.75, 0.9]) AS percentiles
FROM (
    SELECT earn.year, earn.quarter, earn.wage, left(empl.naics::text, 2) naics2
    FROM kcmo_lehd.mo_wage AS earn
    JOIN kcmo_lehd.mo_qcew_employers AS empl
        ON earn.ein = empl.ein 
    WHERE earn.year = 2015 AND earn.quarter = 1 
        AND empl.year = 2015 AND empl.qtr = 1
) subquery
GROUP BY naics2
ORDER BY naics2, percentiles
"""
res = pd.read_sql(query, conn)
print('query took {:.1f} seconds'.format(time.time()-start_time))

In [ ]:
res.info()

In [ ]:
# we can see from the count of non-null values in the `naics2` column if 
# there are some missing values in the data
# if there are missing values we can view those rows with the following code

res[res['naics2'].isnull()]

In [ ]:
# DataFrames also have useful functions like pivot tables:
res.pivot_table(values='earnings', columns='percentiles', index='naics2')

> The distribution of wages paid by different industry is one way to explore how the **jobs very across industry groups.**

## Exploring education and training data
[Table of Contents](#Table-of-Contents)


In [ ]:
# Get a reminder of what education tables we have for Missouri

query = '''
SELECT table_name 
FROM information_schema.tables
WHERE table_schema = 'mo_dhe'
'''
pd.read_sql(query, conn)

In [ ]:
# dhe is the MO Higher Ed data, see the columns

query = '''
SELECT column_name 
FROM information_schema.columns
WHERE table_schema = 'mo_dhe'
    AND table_name = 'completions'
'''
dhe_columns = pd.read_sql(query, conn)
dhe_columns

In [ ]:
# let's check the number of records per year
# and confirm if there is a unique SSN per year

start_time = time.time()
query = """
select calyear, count(*) recs, count(distinct deident_id) ind
from mo_dhe.completions 
group by calyear
order by calyear;
"""
print(pd.read_sql(query, conn))
print('query completed in {:.2f} seconds'.format(time.time()-start_time))

> the "ind" count is slightly lower than the total "recs" count, indicating not each SSN is unique per year

### Ohio data
[Table of Contents](#Table-of-Contents)

The OH HEI data is structured in "wide" format, where a given individual has one row with all of their information in it. 

In [ ]:
# first we can read all the columns into a dataframe
query = '''
SELECT column_name 
FROM information_schema.columns
WHERE table_schema = 'data_ohio_olda_2018'
    AND table_name = 'oh_hei'
'''
hei_columns = pd.read_sql(query, conn)
hei_columns

In [ ]:
deg_cols = [c for c in hei_columns['column_name'].values if c.startswith('deg') and c.endswith('_1')]
len(deg_cols)

> This line of code may look complicated, so let's break it down step by step:
>
> 1. __`... for c in hei_columns['column_name'].values ...`__ - Loop through every element `c` in the list `hei_columns['column_name'].values`
> 2. __`... if c.startswith('deg') and c.endswith('_1')`__ - Return only values that start with `deg` and end with `_1`
> 3. __`c ...`__ - return value c in my new list
>
> _Additional Note: The formulation `[<action> for <item> in <iterable>]`is known as "list comprehension"._ 

In [ ]:
# explore 2014 HEI data
# cannot pull 1-year into memory, need subset of columns
start_time = time.time()

sql = """
SELECT *
FROM (SELECT key_id, file_year AS year,
        unnest(array[{col_str}]) AS col_name,
        unnest(array[{col_val}]) AS col_value
    FROM data_ohio_olda_2018.oh_hei 
    WHERE file_year = 2014
    ) sub_query
WHERE col_value IS NOT NULL AND col_value <> ''
""".format(
col_str=','.join(["'"+c+"'" for c in deg_cols]),
col_val=','.join([c+'::text' for c in deg_cols])
)
df = pd.read_sql(sql, conn)
# print(sql)

print('query completed in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# how many people earned at least one degree/certificate in 2014?
df['key_id'].nunique()

In [ ]:
df[df['col_name'].str.contains('subject')].head()

> Subject codes can be looked up in the `data_ohio_olda_2018.oh_cip_to_soc_crosswalk` table; note the CIP codes are formatted `##.####`

In [ ]:
# how many subjects were studied?
df[df['col_name'].str.contains('subject')]['col_value'].nunique()

### Employment and Education
[Table of Contents](#Table-of-Contents)

Now we'll explore how many OH 2014 grads were **employed** in Ohio.

In [ ]:
start_time = time.time()

sql = """
CREATE TEMP TABLE oh_hei_2014 AS
SELECT * 
FROM (SELECT key_id, file_year AS year,
        unnest(array[{col_str}]) AS col_name,
        unnest(array[{col_val}]) AS col_value
    FROM data_ohio_olda_2018.oh_hei 
    WHERE file_year = 2014) sub_query
WHERE col_value IS NOT NULL AND col_value <> ''
""".format(
col_str=','.join(["'"+c+"'" for c in deg_cols]),
col_val=','.join([c+'::text' for c in deg_cols])
)
# print(sql)
conn.execute(sql)

print('query completed in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
# count records and distinct people in temp table to confirm it's the same as we expect
pd.read_sql('SELECT count(*) recs, count(distinct key_id) people from oh_hei_2014', conn)

In [ ]:
# how many of the 2014 graduates were employed in 2015?
start_time = time.time()

sql = """
select count(*) job_qtrs, count(distinct key_id) employees
from data_ohio_olda_2018.oh_ui_wage_by_quarter
where year = 2015
and key_id IN (select distinct key_id from oh_hei_2014 )
"""
print(pd.read_sql(sql, conn))
print('query completed in {:.2f} seconds'.format(time.time()-start_time))

## Creating New Measures
- Back to [Table of Contents](#Table-of-Contents)

> **Questions to consider** (we will revisit similar questions of measurement frequently during the program)
0. What problem are we working to solve? How can we measure it?
1. How should we define that an individual "received a job"? For example, definitions could be 
  * Received greater than 0 pay at some point within 1 year
  * Received greater than minimum wage (assuming XyZ) in 6 of 8 quarters after graduating
2. How narrowly can you define the unit of analysis? Eg an individual who graduated in year Y...
3. What additional information do we know about these individuals?

**Preliminary Examples**

As the notebooks progress we will dig into different aspects of the above questions, but for now we will show an example of defining MO graduates' employment outcomes: employed in MO for the first quarter of the year after graduation

In [ ]:
# create 2014 MO grads dataframe
sql = """
SELECT *
FROM mo_dhe.completions
WHERE calyear = 2014;
"""

df_grads = pd.read_sql(sql, conn)
df_grads.head()

In [ ]:
# read 2015 Q1 jobs into dataframe

start_time = time.time()

sql = """
SELECT *
FROM kcmo_lehd.mo_wage
WHERE year = 2015 
    AND quarter = 1
"""

df_jobs = pd.read_sql(sql, conn)

print('query completed in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
df_jobs.info()

In [ ]:
# number of unique individuals in jobs dataframe
df_jobs['ssn'].nunique()

In [ ]:
# combine grads and jobs dataframe to grads dataframe on ID/SSN columns
df = pd.merge(df_grads, df_jobs, left_on = 'deident_id', right_on = 'ssn')
df.head()

In [ ]:
print('{:,.0f} of {:,.0f} 2014 graduates are present in MO 2015 wage data'.format(df['ssn'].nunique(), 
                                                                             df_grads['deident_id'].nunique()))

In [ ]:
# what are the distribution of earnings?
df['wage'].describe(percentiles=[.1, .25, .5, .75, .9])

In [ ]:
# how many graduates made more than $3,000 in each of 2 or more quarters?
sum(df_jobs[df_jobs['wage']>3000].groupby('ssn')['id'].count() > 1)

__Separate example: Replicating the QWI Statistics__

The QWI Statistics notebook demonstrates another example of feature creation: the Quarterly Workforce Indicators Census framework using IL wage records.